In [4]:
import numpy as np
import open3d as o3d

In [5]:
pcd = o3d.io.read_point_cloud("./clean points/dwnsample_pose_1.ply")
o3d.visualization.draw_geometries([pcd])

In [ ]:
#downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd.estimate_normals()
pcd.orient_normals_towards_camera_location(camera_location=np.asarray([0., 0., 0.]))
o3d.visualization.draw_geometries([pcd])

In [3]:
def getCenterPoint(points_list):
    count = len(points_list)
    #init
    center = np.zeros(3)
    for i in range(count):
        center[0] += points_list[i][0]
        center[1] += points_list[i][1]
        center[2] += points_list[i][2]
    return center/count

In [2]:
getCenterPoint(np.asarray(pcd.points))

array([1. , 0.5, 2. ])

In [ ]:
def duplicateSurface(numny_array_points):
    temp = []
    for i in numny_array_points:
        #temp.append(i)
        temp.append([i[0],i[1],i[2]+0.05])
    return np.asarray(temp)

In [ ]:
temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
#pcd.paint_uniform_color([1, 0, 0])
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)

In [ ]:
print('run Poisson surface reconstruction')
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(new_pcl, depth=8)
print(mesh)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

In [ ]:
# pcd.estimate_normals()
# distances = pcd.compute_nearest_neighbor_distance()
# avg_dist = np.mean(distances)
# radius = 4*avg_dist
# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd,
#     o3d.utility.DoubleVector([radius,radius*2]))

In [ ]:
o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/pose_1.ply',mesh)